<a href="https://www.kaggle.com/code/aleksandrmorozov123/nlp-with-python?scriptVersionId=152594765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.REL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.QRY


**Code to populate the documents dictionary**

In [2]:
def read_documents ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL")
    merged = " "
    # the string variable merged keeps the result of merging the field identifier with its content
    
    for a_line in f.readlines ():
        if a_line.startswith ("."):
            merged += "\n" + a_line.strip ()
        else:
            merged += " " + a_line.strip ()
    # updates the merged variable using a for-loop
    
    documents = {}
    
    content = ""
    doc_id = ""
    # each entry in the dictioanry contains key = doc_id and value = content
    
    for a_line in merged.split ("\n"):
        if a_line.startswith (".I"):
            doc_id = a_line.split (" ") [1].strip()
        elif a_line.startswith (".X"):
            documents[doc_id] = content
            content = ""
            doc_id = ""
        else:
            content += a_line.strip ()[3:] + " "
    f.close ()
    return documents

# print out the size of the dictionary and the content of the very first article
documents = read_documents ()
print (len (documents))
print (documents.get ("1"))
    

1460
 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


**Code to populate the queries dictionary**

In [3]:
def read_queries ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.QRY")
    merged = ""
    
    # merge the conten of each field with its identifier and separate different fields with lune breaks
    for a_line in f.readlines ():
        if a_line.startswith ("."):
            merged += "\n" + a_line.strip ()
        else:
            merged += " " + a_line.strip ()
    
    queries = {}
    
    # initialize queries dictionary with key = qry_id and value=content for each query in the dataset
    content = ""
    qry_id = ""
    
    for a_line in merged.split ("\n"):
        if a_line.startswith (".I"):
            if not content == "":
                queries [qry_id] = content
                content = ""
                qry_id = ""
            # add an enrty to the dictionary when you encounter an .I identifier
            qry_id = a_line.split(" ")[1].strip ()
        # otherwise, keep adding content to the content variable
        elif a_line.startswith (".W") or a_line.startswith (".T"):
            content += a_line.strip ()[3:] + " "
    queries [qry_id] = content
    f.close ()
    return queries

# print out the length of the dictionary and the content of the first query
queries = read_queries ()
print (len (queries))
print (queries.get("1"))

112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles? 


**Code to populate the mappings dictionary**

In [4]:
def read_mappings ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.REL")
    mappings = {}
    
    for a_line in f.readlines ():
        voc = a_line.strip ().split ()
        key = voc[0].strip ()
        current_value = voc[1].strip()
        value = []
        # update the entry in the mappings dictionary with the current value
        if key in mappings.keys ():
            value = mappings.get (key)
        value.append (current_value)
        mappings [key] = value
    f.close ()
    return mappings

# print out some information about the mapping data structure
mappings = read_mappings ()
print (len (mappings))
print (mappings.keys ())
print (mappings.get ("1"))

76
dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '37', '39', '41', '42', '43', '44', '45', '46', '49', '50', '52', '54', '55', '56', '57', '58', '61', '62', '65', '66', '67', '69', '71', '76', '79', '81', '82', '84', '90', '92', '95', '96', '97', '98', '99', '100', '101', '102', '104', '109', '111'])
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


**Preprocess the data in documents and queries**

In [5]:
# import required libraries
import nltk
from nltk import word_tokenize

# text is converted to lowercase and split into words
def get_words (text):
    word_list = [word for word in word_tokenize (text.lower ())]
    return word_list
    
doc_words = {}
qry_words = {}

for doc_id in documents.keys ():
    doc_words [doc_id] = get_words (documents.get (doc_id))
for qry_id in queries.keys ():
    # entries in both documents and queries are represented as word lists
    qry_words [qry_id] = get_words (queries.get (qry_id))
    
# print out the length of the dictionaries and check the first document and the fisrt query
print (len (doc_words))
print (doc_words.get ("1"))
print (len (doc_words.get ("1")))
print (len (qry_words))
print (qry_words.get ("1"))
print (len (qry_words.get("1")))

1460
['18', 'editions', 'of', 'the', 'dewey', 'decimal', 'classifications', 'comaromi', ',', 'j.p.', 'the', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'dewey', 'decimal', 'classification', '.', 'the', 'first', 'edition', 'of', 'the', 'ddc', 'was', 'published', 'in', '1876', ',', 'the', 'eighteenth', 'edition', 'in', '1971', ',', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed', '.', 'in', 'spite', 'of', 'the', 'ddc', "'s", 'long', 'and', 'healthy', 'life', ',', 'however', ',', 'its', 'full', 'story', 'has', 'never', 'been', 'told', '.', 'there', 'have', 'been', 'biographies', 'of', 'dewey', 'that', 'briefly', 'describe', 'his', 'system', ',', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad', '.']
113
112
['what', 'problems', 'and', 'concerns',

**Simple Biilean search algorithm**

In [6]:
# iterate through the documents
def retrieve_documents (doc_words, query):
    docs = []
    for doc_id in doc_words.keys ():
        found = False
        i = 0
        while i<len(query) and not found: 
            word = query [i]
            if word in doc_words.get (doc_id):
                docs.append (doc_id)
                found = True
            else:
                i+=1
    return docs

# check the results
docs = retrieve_documents (doc_words, qry_words.get("3"))
print (docs [:100])
print (len (docs))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102']
1397


**Begin the preprocessing - remove stopwords and punctuation marks**

In [7]:
# import python's string module that will help remove punctuation marks
import string

# import the stopwords list
from nltk import word_tokenize
from nltk.corpus import stopwords

def process (text):
    stoplist = set (stopwords.words ('english'))
    # only add tthe words if they are not included in the stoplist and are not puctuation marks
    word_list = [word for word in word_tokenize (text.lower())
                if not word in stoplist and not word in string.punctuation]
    return word_list

# check the results of these preprocessing steps on some documents or queries
word_list = process (documents.get ("1"))
print (word_list)

['18', 'editions', 'dewey', 'decimal', 'classifications', 'comaromi', 'j.p.', 'present', 'study', 'history', 'dewey', 'decimal', 'classification', 'first', 'edition', 'ddc', 'published', '1876', 'eighteenth', 'edition', '1971', 'future', 'editions', 'continue', 'appear', 'needed', 'spite', 'ddc', "'s", 'long', 'healthy', 'life', 'however', 'full', 'story', 'never', 'told', 'biographies', 'dewey', 'briefly', 'describe', 'system', 'first', 'attempt', 'provide', 'detailed', 'history', 'work', 'spurred', 'growth', 'librarianship', 'country', 'abroad']


**Next step in preprocessing - stemming**

In [8]:
# import the stemmer
from nltk.stem.lancaster import LancasterStemmer

def process (text):
    stoplist = set (stopwords.words ('english'))
    # initialize the LancasterStemmer
    st = LancasterStemmer ()
    word_list = [st.stem(word) for word in word_tokenize (text.lower ())
                if not word in stoplist and not word in string.punctuation]
    return word_list

# check the results on some document, query, or on a list of words
word_list = process (documents.get("26"))
print (word_list)
word_list = process ("organize, organizing, organizational, organ, organic, organizer")
print (word_list)

['index', 'abstract', 'assocy', 'doyl', 'l.b', 'artic', 'discuss', 'poss', 'exploit', 'stat', 'word', 'co-occurrence', 'text', 'purpos', 'docu', 'retriev', 'co-occurrence', 'defin', 'rel', 'ment', 'process', 'auth', 'read', 'sev', 'mean', 'quantit', 'meas', 'word', 'co-occurrence', 'scrutinized', 'shown', 'strongly', 'co-occurring', 'word', 'pair', 'theref', '``', 'assocy', "''", 'stat', 'sens', 'repres', 'form', '``', 'assocy', 'map', "''", 'last', 'half', 'artic', 'pres', 'two', 'mod', 'us', 'assocy', 'map', 'lit', 'search']
['org', 'org', 'org', 'org', 'org', 'org']


**Estimate term frequency in documents and queries**

In [9]:
def get_terms (text):
    stoplist = set (stopwords.words ('english'))
    terms = {}
    st = LancasterStemmer ()
    word_list = [st.stem(word) for word in word_tokenize (text.lower ())
                if not word in stoplist and not word in string.punctuation]
    for word in word_list:
        terms [word] = terms.get (word, 0) + 1
    return terms

doc_terms = {}
qry_terms = {}
for doc_id in documents.keys ():
    doc_terms [doc_id] = get_terms (documents.get (doc_id))
for qry_id in queries.keys ():
    # populate the term frequency dictionaries for all documents and all queries
    qry_terms [qry_id] = get_terms (queries.get (qry_id))
    
# check the results
print (len (doc_terms))
print (doc_terms.get ("1"))
print (len (doc_terms.get("1")))
print (len (qry_terms))
print (qry_terms.get("1"))
print (len (qry_terms.get("1")))


1460
{'18': 1, 'edit': 4, 'dewey': 3, 'decim': 2, 'class': 2, 'comarom': 1, 'j.p.': 1, 'pres': 1, 'study': 1, 'hist': 2, 'first': 2, 'ddc': 2, 'publ': 1, '1876': 1, 'eighteen': 1, '1971': 1, 'fut': 1, 'continu': 1, 'appear': 1, 'nee': 1, 'spit': 1, "'s": 1, 'long': 1, 'healthy': 1, 'lif': 1, 'howev': 1, 'ful': 1, 'story': 1, 'nev': 1, 'told': 1, 'biograph': 1, 'brief': 1, 'describ': 1, 'system': 1, 'attempt': 1, 'provid': 1, 'detail': 1, 'work': 1, 'spur': 1, 'grow': 1, 'libr': 1, 'country': 1, 'abroad': 1}
43
112
{'problem': 1, 'concern': 1, 'mak': 1, 'describ': 1, 'titl': 3, 'difficul': 1, 'involv': 1, 'autom': 1, 'retriev': 1, 'artic': 2, 'approxim': 1, 'us': 1, 'relev': 1, 'cont': 1}
14


**Code to represent the datya in a shared space**

In [10]:
# collect the shared vocabulary of terms from documents and queries and return it as a sorted list
def collect_vocabulary ():
    all_terms = []
    for doc_id in doc_terms.keys ():
        for term in doc_terms.get (doc_id).keys():
            all_terms.append (term)
    for qry_id in qry_terms.keys ():
        for term in qry_terms.keys():
            for term in qry_terms.get(qry_id).keys():
                all_terms.append (term)
    return sorted (set (all_terms))

# print out the length of the shared vocabulary and check the first several terms in the vocabulary
all_terms = collect_vocabulary ()
print (len (all_terms))
print (all_terms [:10])

def vectorize (input_features, vocabulary):
    output = {}
    for item_id in input_features.keys ():
        features = input_features.get (item_id)
        output_vector = []
        for word in vocabulary:
            if word in features.keys ():
                output_vector.append (int (features.get (word)))
            else:
                output_vector.append (0)
        output [item_id] = output_vector
    return output

doc_vectors = vectorize (doc_terms, all_terms)
qry_vectors = vectorize (qry_terms, all_terms)

# print out some statistics on these data structures
print (len (doc_vectors))
print (len (doc_vectors.get ("1450")))
print (len (qry_vectors))
print (len (qry_vectors.get ("110")))

8881
["''", "'60", "'70", "'a", "'anyhow", "'apparent", "'b", "'basic", "'better", "'bibliograph"]
1460
8881
112
8881


**Calculate and apply inverse document frequency weighting**

In [11]:
# import library for math
import math

def calculate_idfs (vocabulary, doc_features):
    doc_idfs = {}
    for term in vocabulary:
        doc_count = 0
        for doc_id in doc_features.keys ():
            terms = doc_features.get (doc_id)
            if term in terms.keys ():
                doc_count += 1
        doc_idfs [term] = math.log (float (len (doc_features.keys ()))/
                                    float (1 + doc_count), 10)
    return doc_idfs

# check the results - we should have idf values for all terms from the vocabulary
doc_idfs = calculate_idfs (all_terms, doc_terms)
print (len (doc_idfs))
print (doc_idfs.get ("system"))

# define a function to apply idf weighing to the input_terms data structure
def vectorize_idf (input_terms, input_idfs, vocabulary):
    output = {}
    for item_id in input_terms.keys ():
        terms = input_terms.get (item_id)
        output_vector = []
        for term in vocabulary:
            if term in terms.keys ():
                # multiply the term frequencies with idf weights if the term is present in document
                output_vector.append (
                input_idfs.get (term) * float (terms.get (term)))
            else:
                output_vector.append (float (0))
        output [item_id] = output_vector
    return output

# apply idf weighing to doc_terms
doc_vectors = vectorize_idf (doc_terms, doc_idfs, all_terms)

# print out some statistics, such as the number of documents and terms
print (len (doc_vectors))
print (len (doc_vectors.get ("1460")))

8881
0.43844122348938885
1460
8881
